<a href="https://colab.research.google.com/github/andresalcaraaz/Netflix-Project-2-ETL-Data/blob/master/Sonya_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [75]:
# Start Spark session
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [76]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filterNames.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("filterNames.csv"), sep=",", header=True)

# Show DataFrame
df.createOrReplaceTempView('names')

In [77]:
#spark.sql("SELECT * FROM names WHERE primaryprofession = 'actor' OR primaryprofession = 'actress' ").show()



df2 = spark.sql("SELECT * FROM names WHERE primaryprofession in ('actor', 'actress') ")

clean_professions_df = df2.drop("birthyear","knownfortitles","knownFor","deathyear")

clean_professions_df.show()

+---------+--------------------+-----------------+
|   nconst|         primaryname|primaryprofession|
+---------+--------------------+-----------------+
|nm0458601|          Mark Kleid|            actor|
|nm0458664|          Bart Klein|            actor|
|nm0458866|        Leslie Klein|          actress|
|nm0458929|         Paula Klein|          actress|
|nm0459041|     Kevin Kleinberg|            actor|
|nm0459062| Adelheid Kleineidam|          actress|
|nm0459073|      Sergio Kleiner|            actor|
|nm0459084|  Franziska Kleinert|          actress|
|nm0459106|   Stefan Kleinkrieg|            actor|
|nm0459132|    Manny Kleinmuntz|            actor|
|nm0459186|      Maria Klejdysz|          actress|
|nm0459299|     Ladislav Klepal|            actor|
|nm0459318|      Kamaljeet Kler|          actress|
|nm0459364|        Aaron Kleven|            actor|
|nm0459522|      Nele Savicenko|          actress|
|nm0459580|        Atila Klince|            actor|
|nm0459683|       Kent Klineman

In [80]:
df.createOrReplaceTempView('names1')

In [81]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filteredPrincipals.csv"
spark.sparkContext.addFile(url)
df3 = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

# Show DataFrame
df3.show()

+----------+--------+---------+---------------+----+-----------------+
|    tconst|ordering|   nconst|       category| job|       characters|
+----------+--------+---------+---------------+----+-----------------+
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       6|nm5943320|          actor|null|      "[""Lee""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12189034|       8|nm7672770|          actor|null|"[""Detective""]"|
|tt12190374|       3|nm0767339|cinematographer|null|             null|
|tt12190374|       3|nm0767339|cinematographer|null|             null|
|tt12190374|       3|nm0767339|cinematographer|null|             null|
|tt121

In [82]:
# Show DataFrame
df3.createOrReplaceTempView('names2')

In [83]:


df3 = spark.sql("SELECT * FROM names2 WHERE names2.category in ('actor', 'actress') ")

clean_characters_df = df3.drop("tconst","ordering","job")

clean_characters_df.show()

+---------+--------+-----------------+
|   nconst|category|       characters|
+---------+--------+-----------------+
|nm5943320|   actor|      "[""Lee""]"|
|nm5943320|   actor|      "[""Lee""]"|
|nm5943320|   actor|      "[""Lee""]"|
|nm5943320|   actor|      "[""Lee""]"|
|nm7672770|   actor|"[""Detective""]"|
|nm7672770|   actor|"[""Detective""]"|
|nm7672770|   actor|"[""Detective""]"|
|nm7672770|   actor|"[""Detective""]"|
|nm2698001| actress|             null|
|nm2698001| actress|             null|
|nm2698001| actress|             null|
|nm2698001| actress|             null|
|nm8435168|   actor|    "[""Quavo""]"|
|nm8435168|   actor|    "[""Quavo""]"|
|nm8435168|   actor|    "[""Quavo""]"|
|nm8435168|   actor|    "[""Quavo""]"|
|nm3559357| actress|             null|
|nm3559357| actress|             null|
|nm3559357| actress|             null|
|nm3559357| actress|             null|
+---------+--------+-----------------+
only showing top 20 rows



In [84]:
clean_characters_df.join(clean_professions_df,clean_characters_df.nconst ==  clean_professions_df.nconst,"outer") \
     .show(truncate=False)

+---------+--------+--------------------------+------+-----------+-----------------+
|nconst   |category|characters                |nconst|primaryname|primaryprofession|
+---------+--------+--------------------------+------+-----------+-----------------+
|nm0000198|actor   |"[""Solomon""]"           |null  |null       |null             |
|nm0000198|actor   |"[""Solomon""]"           |null  |null       |null             |
|nm0000198|actor   |"[""Solomon""]"           |null  |null       |null             |
|nm0000198|actor   |"[""Solomon""]"           |null  |null       |null             |
|nm0000198|actor   |"[""Shelly Runyon""]"     |null  |null       |null             |
|nm0000198|actor   |"[""Shelly Runyon""]"     |null  |null       |null             |
|nm0000198|actor   |"[""Shelly Runyon""]"     |null  |null       |null             |
|nm0000198|actor   |"[""Shelly Runyon""]"     |null  |null       |null             |
|nm0000198|actor   |"[""Nathaniel Shepherd""]"|null  |null       

In [86]:
#clean_professions_df[['nconst','category']].merge(clean_characters_df[['characters','primaryname']], on='nconst')
#df[['col1', 'col2']].merge(df2[['col1', 'col3']], on='col1')
new_professions_df=clean_characters_df.join(clean_professions_df,"nconst")